Problem: Data is in excel and you need json objects as input

Solution: The below code converts Excel rows to a JSON string using python pandas lib

In [2]:
# import json
# import pandas

# excel_data_Enrol = pandas.read_excel('filename.xlsx', sheet_name='Enrolment')
# excel_data_Acad = pandas.read_excel('filename.xlsx', sheet_name='Academics')

# json_str_Enrol = excel_data_Enrol.to_json(orient='records')
# json_str_Acad = excel_data_Acad.to_json(orient='records')

# print('Enrolment Data to JSON:\n', json_str_Enrol, '\n\n')
# print('Academic Data to JSON:\n', json_str_Acad, '\n\n')

# data = {"analytics":[{"enrolment":json.loads(json_str_Enrol)},{"academics":json.loads(json_str_Acad)}]}
# json_data = json.dumps(data)
# # Write the data to a text file
# with open('output.txt', 'w') as f:
#         f.write(json_data)

import json
import pandas

excel_data_Counties = pandas.read_excel('filename.xlsx', sheet_name='counties')
excel_data_Constituencies = pandas.read_excel('filename.xlsx', sheet_name='constituencies')

json_str_Counties = excel_data_Counties.to_json(orient='records')
json_str_Constituencies = excel_data_Constituencies.to_json(orient='records')

print('Counties Data to JSON:\n', json_str_Counties, '\n\n')
print('Constituencies Data to JSON:\n', json_str_Constituencies, '\n\n')

# data = {"counties":[{"enrolment":json.loads(json_str_Counties)},{"constituencies":json.loads(json_str_Constituencies)}]}
# json_data = json.dumps(data)
# Write the data to a text file
with open('counties.json', 'w') as f:
        f.write(json_str_Counties)
with open('constituencies.json', 'w') as f:
        f.write(json_str_Constituencies)

Counties Data to JSON:
 [{"index":1,"name":"Mombasa","population":939370,"constitutuencyCount":6,"wardCount":30},{"index":2,"name":"Kwale","population":649931,"constitutuencyCount":4,"wardCount":20},{"index":3,"name":"Kilifi","population":1109735,"constitutuencyCount":7,"wardCount":35},{"index":4,"name":"Tana-River","population":240075,"constitutuencyCount":3,"wardCount":15},{"index":5,"name":"Lamu","population":101539,"constitutuencyCount":2,"wardCount":10},{"index":6,"name":"Taita\/Taveta","population":284657,"constitutuencyCount":4,"wardCount":20},{"index":7,"name":"Garissa","population":623060,"constitutuencyCount":6,"wardCount":30},{"index":8,"name":"Wajir","population":661941,"constitutuencyCount":6,"wardCount":30},{"index":9,"name":"Mandera","population":1025756,"constitutuencyCount":6,"wardCount":30},{"index":10,"name":"Marsabit","population":291166,"constitutuencyCount":4,"wardCount":20},{"index":11,"name":"Isiolo","population":143294,"constitutuencyCount":2,"wardCount":10},{"

AttributeError: 'str' object has no attribute 'counties'

scrap data on mps from http://www.parliament.go.ke/the-national-assembly/mps

In [ ]:
from bs4 import BeautifulSoup
import json
import requests
from urllib.parse import urlparse, parse_qs

root_url = "http://www.parliament.go.ke"

def extract_mp_data(url):
  """
  Extracts data for each Member of Parliament (MP) from a webpage.

  Args:
      url: The URL of the webpage containing the table.

  Returns:
      A list of dictionaries, where each dictionary represents an MP with their details.
  """
  data = []
  profileLinks=[]
  tablePages=[]
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Target the specific nav element
  pager_nav = soup.find('nav', class_='pager', role='navigation')
  if pager_nav:
  # Find the last 'li' element within the 'ul' with class 'pager__items'
    ul_tag = pager_nav.find('ul')
    if ul_tag:
      # Only call find_last if ul_tag is not None
      # print(str(ul_tag))
      li_elements = ul_tag.find_all('li')
      # Check if any 'li' elements were found
      if li_elements:
          href = li_elements[-1].find('a').get('href')# Get the last 'li' element

          # Parse the query parameters
          print('last pagination: ',href)
          # Split the href_value by '='
          # Split the href_value by '='
          parts = href.split('=')

          # Extract the second part (which contains the digits)
          pages = parts[-1]
          for i in range(int(pages)+1):
            tablePages.append(f"{url}?page={i}")
          
          print("pages: ",tablePages)

      else:
          print("No 'li' elements found in the ul")

  for link in tablePages:
    # Your code here (will be executed for each element)
    # print("LINK: ",link)
    response_ = requests.get(link)
    linkSoup = BeautifulSoup(response_.content, 'html.parser')

    # Find the table body (tbody element)
    tbody = linkSoup.find('tbody')

    # Check if tbody exists
    if not tbody:
      continue


    # Extract data from each table row (class: mp)
    for row in tbody.find_all('tr', class_='mp'):
      cells = row.find_all("td")
      mp_data = {}

      # Name cell (assuming second cell)
      name_cell = cells[0]
      if not name_cell.text.strip():
        continue
      mp_data['name'] = name_cell.text.strip()
      # print('name',name_cell.text.strip())

      # County cell (assuming fourth cell)
      image_cell = cells[1]
      anchor_tag = image_cell.find('a')
      img_tag = anchor_tag.find('img')

      if img_tag:  # Check if anchor_tag exists before accessing attributes
        image_url = img_tag.get('src')
      else:
        image_url = None  # Handle cases where no anchor tag is found
      mp_data['photoUrl'] = root_url+image_url

      # County cell (assuming 3rd cell)
      county_cell = cells[2]
      mp_data['county'] = county_cell.text.strip()

      # Constituency cell (4th cell)
      constituency_cell = cells[3]
      mp_data['constituency'] = constituency_cell.text.strip()

      # Party cell (5th cell)
      party_cell = cells[4]
      mp_data['party'] = party_cell.text.strip()

      # Status cell (6th cell)
      if len(cells) >= 6:  # Ensure there are at least 6 cells in the row
        status_cell = cells[5]
        mp_data['status'] = status_cell.text.strip()

      # Add MP data to the list
      data.append(mp_data)

  return data

if __name__ == "__main__":
  # Replace with the actual URL of the members list
  url = "http://www.parliament.go.ke/the-national-assembly/mps"
  mp_data = extract_mp_data(url)
  print("MPs count: ",len(mp_data))
  # Convert data to JSON
  json_data = json.dumps(mp_data, indent=4)
  print("JSON: ",json_data)


    